In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 케라스 튜너

In [1]:
!pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 30.4 MB/s 
     |████████████████████████████████| 1.6 MB 36.2 MB/s 


In [2]:
import tensorflow as tf
from tensorflow.keras import datasets

import keras_tuner as kt

mnist = datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11501568/11490434 [==============================] - 0s 0us/step


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam

def get_model(hp):
    hp_unit = hp.Int('units', min_value = 16, max_value = 512, step = 16)
    hp_lr = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

    model = Sequential()
    model.add(Flatten(input_shape=(28, 28)))
    model.add(Dense(hp_unit, activation='relu'))
    model.add(Dense(hp_unit, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer = Adam(learning_rate = hp_lr),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'],
    )
    return model

In [4]:
kerastuner = kt.Hyperband(get_model,
                          directory = 'my_dir',
                          objective = 'val_accuracy', 
                          max_epochs = 5,
                          factor = 3,
                          project_name = 'kerastuner-mnist')

In [5]:
kerastuner.search(x_train, y_train, 
                  validation_data = (x_test, y_test),
                  epochs = 5)

best_hps = kerastuner.get_best_hyperparameters(num_trials = 1)[0]

print ('best parameter - units : ', best_hps.get('units'))
print ('best parameter - learning_rate : ', best_hps.get('learning_rate'))

Trial 10 Complete [00h 00m 27s]
val_accuracy: 0.9485999941825867

Best val_accuracy So Far: 0.9729999899864197
Total elapsed time: 00h 05m 31s
best parameter - units :  304
best parameter - learning_rate :  0.0001


In [6]:
model = kerastuner.hypermodel.build(best_hps)

history = model.fit(x_train, y_train, 
                    validation_split=0.2,
                    epochs = 5)

loss, accuracy = model.evaluate(x_test, y_test)

print('\naccuracy:', accuracy)
print('loss:', loss)

Epoch 1/5
1500/1500 [==============================] - 9s 6ms/step - loss: 0.4556 - accuracy: 0.8807 - val_loss: 0.2193 - val_accuracy: 0.9379
Epoch 2/5
1500/1500 [==============================] - 8s 6ms/step - loss: 0.1959 - accuracy: 0.9431 - val_loss: 0.1608 - val_accuracy: 0.9549
Epoch 3/5
1500/1500 [==============================] - 8s 6ms/step - loss: 0.1449 - accuracy: 0.9582 - val_loss: 0.1335 - val_accuracy: 0.9626
Epoch 4/5
1500/1500 [==============================] - 8s 6ms/step - loss: 0.1137 - accuracy: 0.9675 - val_loss: 0.1180 - val_accuracy: 0.9651
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.1036 - accuracy: 0.9688

accuracy: 0.9688000082969666
loss: 0.10356929898262024
